In [4]:
# %load excesses.py
import pandas as pd
import numpy as np
idx = pd.IndexSlice

# Columns
#--------------------------------------
#Entity - United States
#Code - USA
#Date - 2020-08-30
#"Deaths, 2020, all ages" - 56575
#"Average deaths, 2015–2019, all ages" - 52515
#deaths_2015_all_ages - 51735
#deaths_2016_all_ages - 51272
#deaths_2017_all_ages - 51272
#deaths_2018_all_ages - 51272
#deaths_2019_all_ages - 51272

# bring in the data in csv form
filename = "excess-mortality-raw-death-count.csv"
data = pd.read_csv(filename)

# make sure the dates are formatted properly
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d")

# set Country and Date as indexes
data = data.set_index(["Entity", "Date"]).sort_index()

# use only 2020 and 2018 data
data = data[["Deaths, 2020, all ages", "deaths_2018_all_ages"]]

# Look at only US data
us = data.loc["United States"]
print(us)

# Drop last 9 rows due to incomplete data to compare for 2020
us.drop(us.tail(9).index, inplace=True)

# show the sum over the columns
print(us.sum(0))



            Deaths, 2020, all ages  deaths_2018_all_ages
Date                                                    
2020-01-05                 60010.0                 66129
2020-01-12                 60545.0                 67491
2020-01-19                 59187.0                 64641
2020-01-26                 58991.0                 62777
2020-02-02                 58649.0                 60972
2020-02-09                 59219.0                 61107
2020-02-16                 58655.0                 59774
2020-02-23                 58701.0                 57790
2020-03-01                 59116.0                 56690
2020-03-08                 59464.0                 57089
2020-03-15                 58465.0                 56317
2020-03-22                 59023.0                 55758
2020-03-29                 62784.0                 54913
2020-04-05                 72052.0                 55285
2020-04-12                 78824.0                 55223
2020-04-19                 7654

/home/megazig/.local/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
